<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение моделей</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Анализ-моделей" data-toc-modified-id="Анализ-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Анализ моделей</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li></ul></div>

# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

**Цель**  

Построить модель для определения стоимости автомобиля. 

**Структура исследования**  

1. Подготовка данных
2. Обучение моделей
3. Анализ моделей

**Использование дополнительных модулей**  

- `pandas` – редактор баз данных  
- `numpy` – работа с многомерными массивами
- `sklearn`, `catboost`, `lightgbm` – машинное обучение

In [1]:
import pandas as pd
import numpy as np
import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

## Подготовка данных

In [2]:
df = pd.read_csv("autos.csv")
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


**Признаки**  
`DateCrawled` – дата скачивания анкеты из базы  
`VehicleType` – тип автомобильного кузова  
`RegistrationYear` – год регистрации автомобиля  
`Gearbox` – тип коробки передач  
`Power` – мощность (л.с.)  
`Model` – модель автомобиля  
`Kilometer` – пробег (км)  
`RegistrationMonth` – месяц регистрации автомобиля  
`FuelType` – тип топлива  
`Brand` – марка автомобиля  
`NotRepaired` – была машина в ремонте или нет  
`DateCreated` – дата создания анкеты  
`NumberOfPictures` – количество фотографий автомобиля  
`PostalCode` – почтовый индекс владельца анкеты (пользователя)  
`LastSeen` – дата последней активности пользователя  

**Целевой признак**  
`Price` – цена (евро)  

In [3]:
# Общие сведения о датафрейме
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


В перечне представленных данных присутствуют признаки, которые напрямую не оказывают влияния на конечную стоимость автомобиля. Удаляем эти столбцы.

In [4]:
df = df.drop(columns=['DateCrawled', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen'])

Пропуски в столбцах `VehicleType`, `Gearbox`, `Model`, `FuelType` и `NotRepaired`  заполним значением `missing`.

In [5]:
df[['VehicleType', 'Gearbox', 'Model', 'FuelType', 'NotRepaired']] = df[['VehicleType',
                                                                         'Gearbox', 'Model',
                                                                         'FuelType', 'NotRepaired']].fillna('missing')

In [6]:
# Удаляем дубликаты
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

In [7]:
# Получение характеристик числовых величин
df.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth
count,326826.000000,326826.000000,326826.000000,326826.000000,326826.000000
mean,4403.747330,2004.230985,110.244705,128144.073605,5.696239
std,4518.209808,91.120018,195.886373,37947.663920,3.724122
min,0.000000,1000.000000,0.000000,5000.000000,0.000000
25%,1000.000000,1999.000000,69.000000,125000.000000,3.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000
75%,6350.000000,2008.000000,141.000000,150000.000000,9.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000


В столбце `RegistrationYear` содержатся искаженные данные для предельных значений `min` и `max`. В учебных целях ограничим диапазон: нижний предел – 1980 год, верхний предел – 2019 год (максимальное приемлемое значение для обозначения года).

Для столбца `Power` удалим все автомобили с мощностью двигателя свыше 2000 л.с.

Другие столбцы, в которых минимальное значение `min` может быть сомнительным:  
- `Price` минимальное значение `0` может быть указано ввиду того, что транспортное средство таким образом могут сдавать на утилизацию (металлолом и т.д.).
- `Power` автомобиль может продаваться без двигателя.  
- `RegistrationMonth` минимальное значение `0` в данной категории, скорее всего, означает отсутствие данных на момент заполнения карточки на транспортное средство. Значительный пробег у таких автомобилей не позволяет в данном случае присвоить категорию `первый месяц`.

In [8]:
# Корректировка столбца 'RegistrationYear' и 'Power'
df = df[(df['RegistrationYear'] >= 1980) &
        (df['RegistrationYear'] <= 2019) &
        (df['Power'] <= 2000)]

In [9]:
# Выделим столбцы с признаками и метками (целевым признаком)
X = df.drop(['Price'], axis=1)
y = df['Price']

In [10]:
# Разделим датафрейм на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12345)

In [11]:
# Выделим столбцы с числовыми признаками
numerical_columns = ['RegistrationYear', 'Power', 'Kilometer', 'RegistrationMonth']

# Выделим столбцы с категориальными признаками
categorical_columns = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']

In [12]:
# Создаем преобразователь (transformer) столбцов:
# - для столбцов с числовыми признаками выполняем стандартизацию
# - для столбцов с категориальными признаками применяем метод One-Hot Encoding
column_trans = make_column_transformer((StandardScaler(), numerical_columns), 
                                       (OneHotEncoder(), categorical_columns))

### Вывод

Выполнена предварительная обработка предоставленного датафрейма: удалены дубликаты, откорректированы столбцы с аномальными значениями, данные разбиты на подвыборки, создан преобразователь для категориальных и числовых столбцов. 

Данные подготовлены для дальнейшей работы с моделями машинного обучения.

## Обучение моделей

Используются следующие модели: `LinearRegression()`, `CatBoostRegressor()` и `LGBMRegressor()`

In [13]:
# Создаем функцию для оценки показателей моделей машинного обучения
def model_estimation(model):
    """
    Функция принимает на вход модель для машинного обучения
    
    возвращает: RMSE, время обучения и время предсказания модели
    """
    
    current_model = model

    pipe_model = make_pipeline(column_trans, current_model) # создаем конвейер для обработки

    start_l = time.time()
    pipe_model.fit(X_train, y_train) # выполняем обучение
    end_l = time.time()

    exec_time_l = end_l - start_l # оценка времени обучения

    start_p = time.time()
    y_pred = pipe_model.predict(X_test) # выполняем предсказание
    end_p = time.time()

    exec_time_p = end_p - start_p # оценка времени предсказания

    rmse = np.sqrt(mean_squared_error(y_test, y_pred)) # вычисление контрольной метрики
    
    return round(rmse, 2), round(exec_time_l, 2), round(exec_time_p, 2)

In [14]:
# Последовательно вызываем созданную функцию для выбранных моделей

# LinearRegression()
rmse_lr, exec_time_l_lr, exec_time_p_lr = model_estimation(LinearRegression())

# CatBoostRegressor()
rmse_cb, exec_time_l_cb, exec_time_p_cb = model_estimation(CatBoostRegressor(silent=True, random_state=12345))

# LGBMRegressor()
rmse_lgbm, exec_time_l_lgbm, exec_time_p_lgbm = model_estimation(LGBMRegressor(random_state=12345))

### Вывод

Для сравнения были выбраны три модели регрессии машинного обучения. Создана функция для обработки каждой модели из представленных. 

## Анализ моделей

In [15]:
# Соберем полученные данные в один датафрейм
pivot_table = pd.DataFrame(data = {'LinearRegression': [rmse_lr, exec_time_l_lr, exec_time_p_lr], 
                                   'CatBoostRegressor': [rmse_cb, exec_time_l_cb, exec_time_p_cb], 
                                   'LGBMRegressor': [rmse_lgbm, exec_time_l_lgbm, exec_time_p_lgbm]},
                           index = ['RMSE', 'Время обучения', 'Время предсказания'])


pivot_table

,LinearRegression,CatBoostRegressor,LGBMRegressor
RMSE,2654.91,1768.85,1776.20
Время обучения,66.24,121.97,19.60
Время предсказания,0.17,0.36,0.62


### Вывод

Минимальные и очень близкие по значениям показатели контрольной метрики `RMSE` фиксируются у моделей, которые используют градиентный бустинг `CatBoostRegressor` и `LGBMRegressor`. Следует отметить, что по суммарному времени выполнения обучения и предсказания для предоставленных данных лидирует модель `LGBMRegressor`.

**Данные для времени обучения/предсказания при запуске через удаленный сервер меняются. В среднем минимальное время обучения у `LGBMRegressor`**

Таким образом, модель `LGBMRegressor` является оптимальной для решения поставленной задачи.